In [122]:
import torch
import torch.nn as nn
import numpy as np
import math

In [108]:
with open("data/sonar.all-data") as all_data_file:
    lines = all_data_file.readlines()
    all_data = []
    labels = []
    for line in lines:
        line = line.strip().split(',')
        label = line.pop()
        #line = np.asarray(line, dtype=float)
        if label == "R":
            labels.append(0)
            all_data.append(line)
        elif label == "M":
            labels.append(1)
            all_data.append(line)
        else:
            pass
    all_data = np.asarray(all_data, dtype=float)
    labels = np.asarray(labels, dtype=float)


In [109]:
def normalise_2darray(d2array):
    output_array = []
    for array in d2array:
        x = (array - np.mean(array)) / np.std(array)
        x[x<0] *= -1
        x = (x-np.min(x))/(np.max(x) - np.min(x))
        output_array.append(x)
    return np.asarray(output_array)
x_norm = normalise_2darray(all_data)

In [110]:
def train_test_split(input_data, input_labels):
    indices = np.random.permutation(input_data.shape[0])
    split_idx = math.floor(input_data.shape[0] * 0.7)
    train_idx, test_idx = indices[:split_idx], indices[split_idx:]
    train_data, test_data = input_data[train_idx,:], input_data[test_idx,:]
    train_labels, test_labels = input_labels[train_idx], input_labels[test_idx]
    return train_data, test_data, train_labels, test_labels

In [111]:
corr_arr = []
for idx, row in enumerate(x_norm):
    arr = np.asarray(list(row) + [labels[idx]])
    corr_arr.append(arr)
corr_arr = np.asarray(corr_arr)
corr_map = np.corrcoef(corr_arr, rowvar=False).round(2)
corr_map = corr_map[:, 60]  # keep only final column of the heatmap | correlation to target class
corr_map = corr_map.reshape((61, 1))

In [112]:
to_drop = []
for idx, value in enumerate(corr_map):
    if value > -0.1 and value < 0.1:
        to_drop.append(idx)
to_drop

[3,
 4,
 7,
 12,
 13,
 14,
 15,
 16,
 17,
 24,
 25,
 26,
 27,
 28,
 29,
 31,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 48]

In [113]:
x_norm = np.delete(x_norm, to_drop, axis=1)
x_norm.shape

(208, 35)

In [114]:
train_data, test_data, train_labels, test_labels = train_test_split(x_norm, labels)
print(train_data.shape, test_data.shape, train_labels.shape, test_data.shape)


(145, 35) (63, 35) (145,) (63, 35)


In [115]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [116]:
train_data = torch.tensor(train_data, dtype=torch.float32).to(device)
test_data = torch.tensor(test_data, dtype=torch.float32).to(device)
train_labels = torch.tensor(train_labels, dtype=torch.float32).reshape(-1,1).to(device)
test_labels = torch.tensor(test_labels, dtype=torch.float32).reshape(-1,1).to(device)

print(train_data.shape, test_data.shape, train_labels.shape, test_labels.shape)

torch.Size([145, 35]) torch.Size([63, 35]) torch.Size([145, 1]) torch.Size([63, 1])


In [117]:
class Network(nn.Module):
    def __init__(self, input_size, output_size) -> None:
        super(Network, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.network = self.__setup_network()

    def __setup_network(self):
        net = nn.Sequential(
            nn.Linear(self.input_size, 128),
            nn.Tanh(),
            nn.Linear(128, 128),
            nn.Sigmoid(),
            nn.Linear(128, 128),
            nn.Tanh(),
            nn.Linear(128, 64),
            nn.Sigmoid(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, 64),
            nn.Sigmoid(),
            nn.Linear(64, 64),
            nn.Tanh(),
            nn.Linear(64, self.output_size),
            nn.Sigmoid(),
        )
        return net
    def forward(self, input):
        return self.network(input)
    
    def fit(self, input, labels, n_epochs, loss_fn, optimizer, batch_size=10):
        for epoch in range(n_epochs):
            start = 0
            #for i in range(math.floor(input.shape[0]/batch_size)):
            #    X_batch = input[start:(start+batch_size)]
            #    Ybatch = labels[start:(start+batch_size)]
            #    start += batch_size
            #    pass
            prediction = self.forward(input)
            loss = loss_fn(prediction, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            print(f"Epoch {epoch}:\tloss:{loss}")
            
    


In [118]:
model = Network(input_size=35, output_size=1).to(device)
loss_fn = nn.BCELoss()
optimizer = torch.optim.RMSprop(model.network.parameters(), lr=0.0025, alpha=0.8)
model.train()
model.fit(input=train_data, labels=train_labels, n_epochs=1000, loss_fn=loss_fn, optimizer=optimizer)

Epoch 0:	loss:0.7022507786750793
Epoch 1:	loss:0.7196912169456482
Epoch 2:	loss:0.6965036988258362
Epoch 3:	loss:0.6933605670928955
Epoch 4:	loss:0.6929945349693298
Epoch 5:	loss:0.692926287651062
Epoch 6:	loss:0.6928898692131042
Epoch 7:	loss:0.692843496799469
Epoch 8:	loss:0.692757785320282
Epoch 9:	loss:0.6925671696662903
Epoch 10:	loss:0.692166268825531
Epoch 11:	loss:0.6918449401855469
Epoch 12:	loss:0.6959132552146912
Epoch 13:	loss:0.7003113031387329
Epoch 14:	loss:0.6894793510437012
Epoch 15:	loss:0.679996907711029
Epoch 16:	loss:0.6712742447853088
Epoch 17:	loss:0.6654189229011536
Epoch 18:	loss:0.6682761311531067
Epoch 19:	loss:0.6643157601356506
Epoch 20:	loss:0.651519775390625
Epoch 21:	loss:0.6429591774940491
Epoch 22:	loss:0.6357972025871277
Epoch 23:	loss:0.6393651366233826
Epoch 24:	loss:0.6417681574821472
Epoch 25:	loss:0.6349773406982422
Epoch 26:	loss:0.6272374987602234
Epoch 27:	loss:0.6170594692230225
Epoch 28:	loss:0.6159290075302124
Epoch 29:	loss:0.6150357127189

In [121]:
model.eval()
print(model.forward(test_data).round())
print(test_labels)

tensor([[0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]], device='cuda:0', grad_fn=<RoundBackward0>)
tensor([[0.],
        [0.],
        [0.],
        [1.],
        [1.],
    

In [129]:
from torchsummary import summary
#summary(model=model, input_size=(None,model.input_size))